# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [4]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [5]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [6]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [7]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [8]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [9]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [10]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [11]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [12]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [13]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [14]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To start, I should gather initial information about DeepLearningAI from the internet to understand their background, key personnel, recent milestones, and specific needs.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/deeplea

Thought: I now know the final answer

Final Answer: DeepLearningAI is an education technology company founded by Andrew Ng, focusing on making AI education accessible globally. The company is headquartered in Palo Alto, California, and has been operational since 2017. Key personnel include Andrew Ng, the founder, and Jonathan Heyne, the Chief Operating Officer. 

DeepLearningAI's recent developments highlight its commitment to advancing AI education and research. They have been actively involved in offering online courses and resources in AI and machine learning, targeting both individuals and enterprises. The company's recent business activities include the publication of "The Batch," which offers AI news and insights, emphasizing how AI agents are improving in handling diverse data types to spot patterns and create value.

Potential areas where our solutions can provide value include enhancing their AI model training capabilities, improving data handling and processing, and offering 

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

**Email 1: Introduction to Collaboration Opportunities**
Subject: Empowering the Next Generation of AI Innovators
Dear Andrew,
I hope this message finds you well. At CrewAI, we are deeply inspired by DeepLearningAI's commitment to making AI education accessible to a global audience. Your recent initiative, 'The Batch,' brilliantly highlights the potential of AI to transform data into meaningful insights.
We at CrewAI specialize in enhancing AI model training capabilities and data processing efficiencies. I believe there is a significant opportunity for us to collaborate in integrating these advanced solutions with your educational platforms. This could not only streamline your operations but also enrich the learning experience for your users.
Looking forward to exploring this possibility with you.
Best regards,
[Your Name]

**Email 2: Detailing Specific Solutions**
Subject: Tailored AI Solutions to Enhance DeepLearningAI's Offerings
Dear Andrew,
Following our previous correspondence, I would like to delve deeper into how CrewAI's solutions can specifically add value to DeepLearningAI. Our state-of-the-art AI training tools can be customized to support the diverse needs of your learners, facilitating more effective learning paths.
Additionally, our data handling frameworks are designed to process complex datasets efficiently, which could greatly benefit your initiatives like 'The Batch' in distilling actionable insights from vast amounts of data.
I am eager to discuss how these technologies can be adapted to fit your needs and further our shared goal of democratizing AI education.
Best regards,
[Your Name]

**Email 3: Proposal for a Pilot Project**
Subject: Proposal for a Collaborative Pilot Project
Dear Andrew,
To demonstrate the potential of a partnership between CrewAI and DeepLearningAI, we propose initiating a pilot project. This project could focus on integrating our AI model training solutions with one of your upcoming courses, aimed at enhancing both the delivery and the application of AI education.
We believe this hands-on approach will not only showcase the immediate benefits of our solutions but also set the stage for a long-term partnership in innovating AI education.
Please let me know a convenient time to discuss this further.
Best regards,
[Your Name]

**Email 4: Conclusion and Next Steps**
Subject: Next Steps Towards Transforming AI Education Together
Dear Andrew,
Thank you for considering our proposals and discussing the potential synergies between CrewAI and DeepLearningAI. We are excited about the prospect of working together to empower aspiring AI professionals across the globe.
Could we schedule a follow-up meeting next week to finalize the details of our collaboration? Your insight as the leader of DeepLearningAI would be invaluable in shaping this partnership to best serve the educational community.
Looking forward to your thoughts and your availability for a further conversation.
Best regards,
[Your Name]